In [ ]:
// Import the required modules
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FsODE"

// ... and open the required modules
open FsODE
open Plotly.NET

Installed Packages FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

http://be150.caltech.edu/2020/content/lessons/02_intro_to_python_for_biological_circuits.html

Lasst uns das folgende System annehmen, in dem Stimulus X die Produktion von Y auslöst und Y wiederrum Z aktiviert.

$$\quad
X → Y → Z
$$

Stellt man dieses System mit den bereits bekannten Formeln für eine Aktivator bedingte Protein Produktion nach, erhält man folgendes:

$$\quad
\beta_y = \frac{\beta_0\frac{y}{K_y}}{1+\frac{A}{K_d}} \quad ??? Hill\ Koeffizient?
$$

Fragen:
1. Wie wird Hill-Gleichung mit Aktivator Gleichung kombiniert
2. Nondimensionalization

In [ ]:
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

modelContext.SetStepSize(0.01)

In [ ]:
let beta, gamma, n_x, n_y, x_0 =
    1.0,
    1.0,
    2.0,
    2.0,
    2.0

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let y' = beta * (x_0**n_x) / (1. + x_0**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| y'; z' |]

let P0 = [| 
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 1000 // n timepoints
    |> SolPoints.memorize 

let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

[
    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y");

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z");
]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")

<!-- Plotly chart will be drawn inside this DIV -->

Passe Stimulus X so an, dass es nciht mehr eine konstante sondern ein Puls der Länge τ ist.



In [ ]:
let x_pulse(t: float, t_0: float, tau: float, x_0: float) =
    if t >= t_0 && t <= t_0 + tau then 
        x_0
    else
        0.0001

let tArr = [|0.0 .. 0.01 .. 10|]

let stimulusArr = tArr |> Array.map (fun t -> x_pulse(t, 1.0, 2.0, 2.0))

Chart.Line(tArr, stimulusArr)
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless x")

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let beta, gamma, n_x, n_y =
    1.0,
    1.0,
    2.0,
    2.0

let t_0, tau, x_0 =
    1.0,
    4.0,
    2.0

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let x_t = x_pulse(t, t_0, tau, x_0)
        let y' = beta * (x_t**n_x) / (1. + x_t**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| 
            y'
            z'
        |]

let P0 = [|
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 1000
    |> SolPoints.memorize 

let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

[

    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y");

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z");

]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let beta, gamma, n_x, n_y =
    1.0,
    1.0,
    2.0,
    2.0

let t_0, tau, x_0 =
    1.0,
    0.7, // !!! This parameter was changed
    2.0

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let x_t = x_pulse(t, t_0, tau, x_0)
        let y' = beta * (x_t**n_x) / (1. + x_t**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| 
            y'
            z'
        |]

let P0 = [|
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 1000
    |> SolPoints.memorize 

let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

[

    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y");

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z");

]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")

<!-- Plotly chart will be drawn inside this DIV -->

Really short pulses and a lesson about scipy.integrate.odeint()
Now, we will take a brief interlude to learn an important lesson about the algorithm of scipy.integrate.odeint() and its use in these applications. We will consider a very brief pulse, 1≤t≤1.05.

Uh oh! Something went wrong, since the Y signal never went up. This exposes an important issue with the algorithm used by scipy.integrate.odeint(). The Hindmarsh-Petzold algorithm uses variable step sizes so that it takes long steps when the system is not changing much and short steps when it is. Therefore, if we have a long period of no changes (leading up to t=1), the step sizes taken by the solver will increase, and we’ll step right over the pulse.

> This does not work with our algorithm. 🤷

They need to split the system into three parts: "before impulse", "during impulse", and "after impulse". We don't.

In [ ]:
let beta, gamma, n_x, n_y =
    1.0,
    1.0,
    2.0,
    2.0

let t_0, tau, x_0 =
    1.0,
    0.05, // !!! This parameter was changed
    2.0

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let x_t = x_pulse(t, t_0, tau, x_0)
        let y' = beta * (x_t**n_x) / (1. + x_t**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| 
            y'
            z'
        |]

let P0 = [|
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take 1000
    |> SolPoints.memorize 

let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

[

    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y");

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z");

]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
/// Returns x value for periodic forcing of amplitude x_0 and frequency f.
let x_periodic(t:float, f: float, x_0: float) =
    x_0 * (1. + sin(f * t))

let tArr = [|0.0 .. 0.01 .. 10|]

let freqArr = tArr |> Array.map (fun t -> x_periodic(t, 5.0, 2.0))

Chart.Line(tArr, freqArr)
|> Chart.withSize(900, 500)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let beta, gamma, n_x, n_y =
    1.0,
    1.0,
    2.0,
    10.0

let f, x_0 =
    0.5,
    2.0

let tMax = 1000.

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let x_t = x_periodic(t, f, x_0)
        let y' = beta * (x_t**n_x) / (1. + x_t**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| 
            // x_t
            y'
            z'
        |]

let P0 = [|
    // 0.0
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take (int tMax)
    |> SolPoints.memorize 

// let impuls = Ps |> SolPoints.toPoints 1
let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

let xPeriodic =
    let tArr = [0.0 .. 0.01 .. (0.01 * tMax)] 
    let xPeriodic = tArr |> Seq.map (fun x -> x_periodic(x, f, x_0))
    let xMax = xPeriodic |> Seq.max
    let normalizedXPeriodic = xPeriodic |> Seq.map (fun x -> x / xMax)
    Seq.zip tArr normalizedXPeriodic

[
    xPeriodic
    |> Chart.Spline
    |> Chart.withTraceInfo("Impuls")
    |> Chart.withLineStyle(Color = Color.fromHex("#bfefd5"))

    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y")
    |> Chart.withLineStyle(Color = Color.fromString("blue"))

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z")
    |> Chart.withLineStyle(Color = Color.fromString("orange"))

]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")
|> Chart.withSize(800, 500)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let beta, gamma, n_x, n_y =
    1.0,
    1.0,
    2.0,
    10.0

let f, x_0 =
    5.0,
    2.0

let tMax = 1000.

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let x_t = x_periodic(t, f, x_0)
        let y' = beta * (x_t**n_x) / (1. + x_t**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| 
            // x_t
            y'
            z'
        |]

let P0 = [|
    // 0.0
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take (int tMax)
    |> SolPoints.memorize 

// let impuls = Ps |> SolPoints.toPoints 1
let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

let xPeriodic =
    let tArr = [0.0 .. 0.01 .. (0.01 * tMax)] 
    let xPeriodic = tArr |> Seq.map (fun x -> x_periodic(x, f, x_0))
    let xMax = xPeriodic |> Seq.max
    let normalizedXPeriodic = xPeriodic |> Seq.map (fun x -> x / xMax)
    Seq.zip tArr normalizedXPeriodic

[
    xPeriodic
    |> Chart.Spline
    |> Chart.withTraceInfo("Impuls")
    |> Chart.withLineStyle(Color = Color.fromHex("#bfefd5"))

    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y")
    |> Chart.withLineStyle(Color = Color.fromString("blue"))

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z")
    |> Chart.withLineStyle(Color = Color.fromString("orange"))

]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")
|> Chart.withSize(800, 500)

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let beta, gamma, n_x, n_y =
    1.0,
    1.0,
    2.0,
    10.0

let f, a = 
    [|
        0.5 
        10. 
    |],
    [|
        0.5
        2.0
    |]

let tMax = 2500.

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->

        let y = P[0] // Change in protein 1 quantity
        let z = P[1] // Change in protein 2 quantity

        let x_t = Array.zip f a |> Array.map (fun (f,a) -> x_periodic(t, f, a)) |> Seq.sum
        let y' = beta * (x_t**n_x) / (1. + x_t**n_x) - y
        let z' = gamma * (y**n_y / (1. + y**n_y) - z) 

        [| 
            y'
            z'
        |]

let P0 = [|
    0.0
    0.0
|]

let Ps = 
    modelContext.OdeInt(
        0., // start time point
        P0,
        dP_dt
    )
    |> SolPoints.take (int tMax)
    |> SolPoints.memorize 

// let impuls = Ps |> SolPoints.toPoints 1
let protein1 = Ps |> SolPoints.toPoints 1
let protein2 = Ps |> SolPoints.toPoints 2

let xPeriodic =
    let tArr = [0.0 .. 0.01 .. (0.01 * tMax)] 
    let xPeriodic = tArr |> Seq.map (fun t -> 
        let sin_sum = Array.zip f a |> Array.map (fun (f,a) -> x_periodic(t, f, a)) |> Seq.sum
        sin_sum
    )
    let xMax = xPeriodic |> Seq.max
    let normalizedXPeriodic = xPeriodic |> Seq.map (fun x -> x / xMax)
    Seq.zip tArr normalizedXPeriodic

[
    xPeriodic
    |> Chart.Spline
    |> Chart.withTraceInfo("Impuls")
    |> Chart.withLineStyle(Color = Color.fromHex("#bfefd5"))

    protein1
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Y")
    |> Chart.withLineStyle(Color = Color.fromString("blue"))

    protein2
    |> Chart.Spline
    |> Chart.withTraceInfo("Protein Z")
    |> Chart.withLineStyle(Color = Color.fromString("orange"))

]
|> Chart.combine
|> Chart.withXAxisStyle("dimensionless time")
|> Chart.withYAxisStyle("dimensionless y,z")
|> Chart.withSize(800, 500)

<!-- Plotly chart will be drawn inside this DIV -->